# 字幕大师

In [2]:
import psycopg2

## 数据库模块

In [3]:
def get_subtitles_by_video_id(video_id):
    conn=psycopg2.connect(database="finance01",user="python01_user02",password="python01_user02@123",host="110.41.115.206",port=8000)
    cur = conn.cursor()
    # 执行 SQL 查询：根据传入的视频ID，获取该视频的字幕信息，按字幕索引顺序排序
    cur.execute("""
        SELECT subtitle_index, start_time, end_time, text
        FROM subtitles
        WHERE video_id = %s 
        ORDER BY subtitle_index
    """, (video_id,))
    # 将查询结果全部取出，返回的是一个列表，每个元素是一个元组(row)
    rows = cur.fetchall()
    cur.close()
    conn.close()
    # 将结果转换成更清晰的结构（字典列表），每条字幕包含索引、开始时间、结束时间和文本内容
    return [{'index': r[0], 'start': r[1], 'end': r[2], 'text': r[3]} for r in rows]

def search_videos_by_keyword(keyword):
    conn=psycopg2.connect(database="finance01",user="python01_user02",password="python01_user02@123",host="110.41.115.206",port=8000)
    cur = conn.cursor()
    # 执行模糊搜索：查找字幕文本中包含关键字的字幕条目
    cur.execute("""
        SELECT DISTINCT video_id, text
        FROM subtitles
        WHERE text ILIKE %s
    """, (f"%{keyword}%",))
    # 获取所有查询结果，每条结果是一个元组 (video_id, text)
    results = cur.fetchall()
    cur.close()
    conn.close()
    # 将结果转换为字典列表格式，便于程序后续处理
    return [{'video_id': r[0], 'text': r[1]} for r in results]

## 连接大模型

In [10]:
import dashscope
import os
import jieba
import re

# 若指定视频编号则返回对应编号
def extract_video_id_from_question(question):
    # 匹配“视频1”“第2个视频”等说法
    match = re.search(r"(视频|第)(\d+)", question)
    if match:
        return int(match.group(2))
    return None

def extract_keywords(question):
    stopwords = {"的", "是", "了", "我", "你", "他", "她", "它", "在", "和", "就", "都", "还", "呢", "吗"}
    words = jieba.lcut(question)
    keywords = [w for w in words if w not in stopwords and len(w) > 1]
    return keywords

# 从问题中提取视频编号
def find_related_video_ids_by_keywords(keywords, max_videos=3):
    matched_ids = set()
    for kw in keywords:
        videos = search_videos_by_keyword(kw)  # 调用你已有的函数搜索视频
        for v in videos:
            matched_ids.add(v['video_id'])  # 将匹配到的视频 ID 添加进集合
            if len(matched_ids) >= max_videos:  # 控制最多返回多少个视频
                return list(matched_ids)
    return list(matched_ids)


# 获取多个字幕片段并拼接
def get_combined_subtitles(video_ids):
    all_subtitles = []
    for vid in video_ids:
        subtitles = get_subtitles_by_video_id(vid)  # 获取每个视频的字幕
        for s in subtitles:
            # 格式示例：[视频2 00:00:01~00:00:04] 货币供应量对经济有重要影响
            all_subtitles.append(f"[视频{vid} {s['start']}~{s['end']}] {s['text']}")
    return "\n".join(all_subtitles)  # 合并为一个整体文本

dashscope.api_key = "sk-fe6f7bb6ea8f4c108b9a76573d307821"

# 根据问题回答
def ask_question(question):
    # 第一步：检查是否明确指定了视频 ID
    specified_id = extract_video_id_from_question(question)
    
    # 第二步：如果指定了视频，就只用那个；否则提取关键词匹配相关视频
    if specified_id is not None:
        video_ids = [specified_id]
    else:
        keywords = extract_keywords(question)
        video_ids = find_related_video_ids_by_keywords(keywords)
    
    if not video_ids:
        return "找不到与问题相关的视频或字幕内容。"
    print(f'问题所涉及的视频有{video_ids}')
    # 第三步：获取上下文字幕内容
    context = get_combined_subtitles(video_ids)

    # 第四步：构造大模型 prompt（告诉模型背景 + 问题）
    prompt = f"""你是一个学习助手。以下是来自一个或多个教学视频的字幕内容，请根据内容回答用户的问题：

【字幕内容】
{context}

【问题】
{question}

请基于字幕内容作答，如无答案，请回答“字幕中未提及相关内容”。
"""

    # 第五步：调用通义千问进行回答
    try:
        response = dashscope.Generation.call(
            model="qwen-turbo",        # 使用通义千问的轻量级模型
            prompt=prompt,             # 提示词（包含上下文 + 问题）
            top_p=0.8,                 # 控制输出多样性
            temperature=0.7            # 控制回答的随机程度
        )
        return response["output"]["text"]  # 返回模型生成的回答
    except Exception as e:
        return f"调用通义千问失败：{e}"

In [14]:
user_question = "北师大最美的女人是谁？"
answer = ask_question(user_question)
print(answer)

找不到与问题相关的视频或字幕内容。
